 # Count And Identify Trees

In [1]:
import rasterio
from rasterio.transform import from_origin
from rasterio.windows import Window
from deepforest import main
from deepforest import utilities
import ffmpeg
import os
import re
import pandas as pd



In [2]:
class SrtParser:
    def __init__(self,path):
        self.path = path
        file_to_read = open(path,'r')
        self.data = file_to_read.read()
        file_to_read.close()
        self.subtitles = pd.DataFrame(columns=['index','start','end','text','latitude','longitude','distance','height','horizontal_speed','vertical_speed'])

    def parseTime(self,time_str):
        # Extract hours, minutes, and seconds from the time string
        hours, minutes, seconds_milliseconds = time_str.split(':')
        seconds, milliseconds = seconds_milliseconds.split(',')
        
        # Convert hours, minutes, and seconds to integers
        hours = int(hours)
        minutes = int(minutes)
        seconds = int(seconds)
        milliseconds = int(milliseconds)
        
        # Return the total number of milliseconds
        return (hours * 3600 + minutes * 60 + seconds) * 1000 + milliseconds


    def parseSubtitle(self,subtitle_str):
        # Split the subtitle string into lines
        lines = subtitle_str.strip().split('\n')
        
        # Extract the index, start, and end times from the first line
        index = int(lines[0])
        start, end = map(self.parseTime, lines[1].split(' --> '))
        
        # Join the remaining lines into a single string
        text = '\n'.join(lines[2:])
        
        # Create and return a Subtitle object
        new_subtitle = pd.DataFrame.from_dict({
            'index': [index],
            'start': [start],
            'end': [end],
            'text': [text],
            'longitude': [float(text.split(',')[4].lstrip('GPS ('))],
            'latitude': [float(text.split(',')[5])],
            'distance': [float(text.split(',')[7].lstrip('D ').rstrip('m'))],
            'height': [float(text.split(',')[8].lstrip('H ').rstrip('m'))],
            'horizontal_speed': [float(text.split(',')[9].lstrip('H.S ').rstrip('m/s'))],
            'vertical_speed': [float(text.split(',')[10].lstrip('V.S ').rstrip('m/s'))]
        })
        self.subtitles = pd.concat([self.subtitles,new_subtitle],ignore_index=True)

    def parseSrt(self,srt_str):
        # Split the srt string into a list of subtitle strings
        subtitles = re.split('\n\n+', srt_str.strip())
        
        # Parse each subtitle string and return a list of Subtitle objects
        for subtitle in subtitles:
            self.parseSubtitle(subtitle)
        return self.subtitles
    
    def parse(self):
        return self.parseSrt(self.data)

In [3]:
def prepareEnv(envName):
    #If a path or filename is provided the env name is the basename of the given filename.
    envName = os.path.basename(envName).split('.')[0]

    if not os.path.exists('data'):
        os.makedirs('data')
    envPath = 'data/%s'%envName
    if not os.path.exists(envPath):
        os.makedirs(envPath)
    srcImgsPath = '%s/srcImgs'%envPath
    if not os.path.exists(srcImgsPath):
        os.makedirs(srcImgsPath)
    geoImgsPath = '%s/geoImgsPath'%envPath
    if not os.path.exists(srcImgsPath):
        os.makedirs(srcImgsPath)

    config = {
        envName: envName,
        envPath: envPath,
        srcImgsPath: srcImgsPath,
        geoImgsPath: geoImgsPath
    }

    return config


In [4]:

def extractMetaFromVideo(videoFileName): 
    config = prepareEnv(videoFileName)

    ffmpeg.input(videoFileName).output('%s/subs.srt'%config.envPath, map = "0:s:0").run(overwrite_output=True)

In [5]:
def createSrcImgs(videoFileName):
    config = prepareEnv(videoFileName)
    ffmpeg.input(videoFileName).output('%s/%s'%(config.srcImgsPath,"src_%05d.png"),r=1).run(overwrite_output=True)

In [6]:
def generateGeoCodeImage(videoFileName,second,latidute,longitude):
    config = prepareEnv(videoFileName)
    src = rasterio.open('%s/src_%05d.png'%(config.srcImgsPath,second))
    width, height = src.width, src.height
    size = min(width, height)

    # Set the metadata for the GeoTIFF
    meta = {
        'driver': 'GTiff',
        'dtype': 'uint8',
        'nodata': 0,
        'width': size,
        'height': size,
        'count': 3,
        'crs': rasterio.crs.CRS.from_epsg(4326),  # Set the CRS to EPSG: 4326 (WGS 84)
        'transform': from_origin(latidute, longitude, 0.1, 0.1)  # Set the latitude and longitude coordinates as the origin
    }

    # Create the GeoTIFF file
    with rasterio.open('%s/geo_%05d.tiff'%(config.geoImgsPath,second), 'w', **meta) as dst:
        x_offset = (width - size) // 2
        y_offset = (height - size) // 2
        window = Window(x_offset, y_offset, size, size)

        # Copy the data from the source image to the GeoTIFF
        dst.write(src.read(window=window))

    src.close()

In [7]:
def predict(videoFileName,second):
    config = prepareEnv(videoFileName)
    path = '%s/geo_%05d.tiff'%(config.geoImgsPath,second)
    r = rasterio.open(path)
    transform = r.transform 
    crs = r.crs
    r.close()

    m = main.deepforest()
    m.use_release(check_release=False)
    df = m.predict_image(path=path)
    gdf = utilities.annotations_to_shapefile(df, transform=transform, crs=crs)
    return gdf

In [8]:
#videoFileName = "/Users/hakanonal/Downloads/temp_video_for_share.mp4"
#extractMetaFromVideo(videoFileName)
#createSrcImgs(videoFileName)